In [17]:
from CNN_setup.datasets.dataset_tools import load_dataset
from CNN_setup.utils.cnn_models_utils import load_model
from CNN_setup.model.CIFAR_CNN import CIFAR_CNN_Classifier
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

In [12]:
test_rotated = load_dataset(r'data\transformed\cifar-only-0',flat=True)
roated_loader = DataLoader(test_rotated,  batch_size=32, shuffle=True)
test_cifar = CIFAR10(root='./data', train=False, download=True)
orig_loader = DataLoader(test_cifar,  batch_size=32, shuffle=True)

Files already downloaded and verified


In [10]:
class GradualDrifttoader:
    def __init__(self, orig_loader, drift_loader, shift_step = 8):
        self.loader1 = iter(orig_loader)
        self.loader2 = iter(drift_loader)
        self.shift_step = shift_step
        self.current_shift = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_shift > 32:
            raise StopIteration

        batch1 = next(self.loader1, None)
        batch2 = next(self.loader2, None)

        if batch1 is None or batch2 is None:
            raise StopIteration

        # Take the first (batch_size - current_shift) samples from batch1
        batch1 = batch1[:32 - self.current_shift]

        # Take the first current_shift samples from batch2
        batch2 = batch2[:self.current_shift]

        # Combine the two batches
        combined_batch = torch.cat([batch1, batch2])

        # Increase the shift for the next batch
        self.current_shift += self.shift_step

        return combined_batch

In [14]:
gradual_loader = GradualDrifttoader(orig_loader = orig_loader, drift_loader = roated_loader, shift_step=8)

In [18]:
model = load_model('trained_models\CNN_cifar_downloaded.torch', CIFAR_CNN_Classifier())